In [1]:
pip uninstall keras-vggface -y

In [2]:
pip install git+https://github.com/YaleDHLab/vggface.git

  Cloning https://github.com/YaleDHLab/vggface.git to /tmp/pip-req-build-0adwgv5q
  Running command git clone --filter=blob:none --quiet https://github.com/YaleDHLab/vggface.git /tmp/pip-req-build-0adwgv5q
  Resolved https://github.com/YaleDHLab/vggface.git to commit c401b5473f68c6421e6efff4166d22b463b9567c
  Preparing metadata (setup.py) ... done
  Created wheel for keras_vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8329 sha256=ac3f1eae8f176267414eca6982cb51d5deb3f1a8ea5361940688b86d662ffefb
  Stored in directory: /tmp/pip-ephem-wheel-cache-5i_791jk/wheels/43/0a/b6/ddc13802b8db14ec6625d111627433f9b80d4005ff7d19fc9a
Successfully built keras_vggface


In [3]:
pip install keras-applications

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.7 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Imports**

In [5]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

from sklearn.metrics import accuracy_score, precision_score, recall_score, recall_score, f1_score

**Mover las imágenes al disco local**

Se copian las imágenes desde Drive al espacio temporal de Colab para que el entrenamiento sea más rápido

In [6]:
import os

# Definimos rutas
zip_path = "/content/drive/MyDrive/FER-2013.zip"
local_root = "/content"  # Se descomprimirá creando la carpeta temp_fer2013 aquí

print("🚀 Copiando ZIP desde Drive...")
!cp "{zip_path}" /content/dataset.zip

print("📂 Descomprimiendo (esto vuela)...")
!unzip -q /content/dataset.zip -d "{local_root}"

# Tus rutas para el modelo siguen siendo las mismas
train_dir = "/content/train"
test_dir  = "/content/test"

print(f"✅ ¡Listo en segundos! Rutas configuradas:\n{train_dir}\n{test_dir}")

🚀 Copiando ZIP desde Drive...
📂 Descomprimiendo (esto vuela)...
✅ ¡Listo en segundos! Rutas configuradas:
/content/train
/content/test


In [7]:
import os

def contar_dataset(directorio, nombre):
    if not os.path.exists(directorio):
        print(f"❌ La carpeta {nombre} no existe en: {directorio}")
        return

    print(f"📊 RECUENTO PARA: {nombre.upper()}")
    print("-" * 30)

    total_fotos = 0
    # Listamos las subcarpetas (las clases: angry, happy...)
    clases = sorted(os.listdir(directorio))

    for clase in clases:
        ruta_clase = os.path.join(directorio, clase)
        # Solo miramos si es carpeta
        if os.path.isdir(ruta_clase):
            cantidad = len(os.listdir(ruta_clase))
            print(f"   • {clase.ljust(10)}: {cantidad} imágenes")
            total_fotos += cantidad

    print("-" * 30)
    print(f"👉 TOTAL {nombre}: {total_fotos} imágenes\n")

# --- EJECUTAMOS EL CONTEO ---
contar_dataset("/content/train", "Entrenamiento (Train)")
contar_dataset("/content/test", "Prueba (Test)")

📊 RECUENTO PARA: ENTRENAMIENTO (TRAIN)
------------------------------
   • angry     : 3995 imágenes
   • disgust   : 436 imágenes
   • fear      : 4097 imágenes
   • happy     : 7215 imágenes
   • neutral   : 4965 imágenes
   • sad       : 4830 imágenes
   • surprise  : 3171 imágenes
------------------------------
👉 TOTAL Entrenamiento (Train): 28709 imágenes

📊 RECUENTO PARA: PRUEBA (TEST)
------------------------------
   • angry     : 958 imágenes
   • disgust   : 111 imágenes
   • fear      : 1024 imágenes
   • happy     : 1774 imágenes
   • neutral   : 1233 imágenes
   • sad       : 1247 imágenes
   • surprise  : 831 imágenes
------------------------------
👉 TOTAL Prueba (Test): 7178 imágenes



# Rutas

In [8]:
train_dir = r"/content/train"
test_dir  = r"/content/test"

IMG_SIZE = (48,48)
IMG_SIZE_VGG = (224,224)
BATCH_SIZE = 32
BATCH_SIZE_VGG = 64
EPOCHS = 50

# Generadores de datos

## CNN en grayscale

In [9]:
# AUMENTO DE DATOS -> SOLO PARA TRAIN
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.1
)

# SIN AUMENTO -> PARA VALIDACIÓN Y TEST
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="training"
)

val_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)

num_classes = train_generator.num_classes
print("Clases detectadas:", train_generator.class_indices)

Found 25841 images belonging to 7 classes.
Found 2868 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Clases detectadas: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


## VGGFace (RGB)

In [10]:
train_datagen_vgg = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.1
)

validation_datagen_vgg = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.1
)

test_datagen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator_vgg = train_datagen_vgg.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE_VGG,
    color_mode="rgb",
    class_mode="categorical",
    batch_size=BATCH_SIZE_VGG,
    subset="training"
)

val_generator_vgg = validation_datagen_vgg.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE_VGG,
    color_mode="rgb",
    class_mode="categorical",
    batch_size=BATCH_SIZE_VGG,
    subset="validation"
)

test_generator_vgg = test_datagen_vgg.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE_VGG,
    color_mode="rgb",
    class_mode="categorical",
    batch_size=BATCH_SIZE_VGG,
    shuffle=False
)

num_classes = train_generator_vgg.num_classes
print("Clases detectadas:", train_generator_vgg.class_indices)

Found 25841 images belonging to 7 classes.
Found 2868 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Clases detectadas: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


# Early stopping

In [11]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=6,
                               restore_best_weights=True
                               )

# CNN simple

In [ ]:
cnn_model = Sequential()

# Bloque 1
cnn_model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(48, 48, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

# Bloque 2
cnn_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

cnn_model.add(Flatten())
cnn_model.add(Dense(num_classes, activation='softmax'))

cnn_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │        64,519 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 83,335 (325.53 KB)

 Trainable params: 83,335 (325.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

cnn_model.save('/content/drive/MyDrive/FER-2013/modelos/cnn_model.keras')

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


808/808 ━━━━━━━━━━━━━━━━━━━━ 111s 135ms/step - accuracy: 0.2615 - loss: 1.7844 - val_accuracy: 0.4121 - val_loss: 1.5948
Epoch 2/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 107s 132ms/step - accuracy: 0.3514 - loss: 1.6508 - val_accuracy: 0.4195 - val_loss: 1.5329
Epoch 3/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 107s 132ms/step - accuracy: 0.3885 - loss: 1.5900 - val_accuracy: 0.4372 - val_loss: 1.4768
Epoch 4/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 144s 135ms/step - accuracy: 0.4050 - loss: 1.5520 - val_accuracy: 0.4603 - val_loss: 1.4177
Epoch 5/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 146s 140ms/step - accuracy: 0.4150 - loss: 1.5192 - val_accuracy: 0.4787 - val_loss: 1.3866
Epoch 6/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 136s 168ms/step - accuracy: 0.4355 - loss: 1.4795 - val_accuracy: 0.4829 - val_loss: 1.3866
Epoch 7/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 158s 195ms/step - accuracy: 0.4466 - loss: 1.4604 - val_accuracy: 0.4955 - val_loss: 1.3538
Epoch 8/50
808/808 ━━━━━━━━━━━━━━━━━━━━ 130s 161ms/step - accuracy: 0.4541 - loss: 1.43

# VGGFace - Extracción de características

In [12]:
vggface_base = VGGFace(
    model="vgg16",
    include_top=False,
    input_shape=(224,224,3)
)

vggface_base.trainable = False

vgg_feature_model = Sequential()

vgg_feature_model.add(vggface_base)
vgg_feature_model.add(GlobalAveragePooling2D())
vgg_feature_model.add(Dense(256, activation="relu"))
vgg_feature_model.add(Dropout(0.5))
vgg_feature_model.add(Dense(num_classes, activation="softmax"))

vgg_feature_model.summary()

58916864/58909280 [==============================] - 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vggface_vgg16 (Functional)      │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,815 (56.64 MB)

 Trainable params: 133,127 (520.03 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
vgg_feature_model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_vgg_feat = vgg_feature_model.fit(
    train_generator_vgg,
    validation_data=val_generator_vgg,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

vgg_feature_model.save('/content/drive/MyDrive/FER-2013/modelos/vgg_feature_model.keras')

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 413s 962ms/step - accuracy: 0.2279 - loss: 4.8861 - val_accuracy: 0.4059 - val_loss: 1.5828
Epoch 2/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 360s 892ms/step - accuracy: 0.3375 - loss: 1.7639 - val_accuracy: 0.4676 - val_loss: 1.4347
Epoch 3/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 360s 890ms/step - accuracy: 0.3950 - loss: 1.5938 - val_accuracy: 0.5007 - val_loss: 1.3431
Epoch 4/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 355s 878ms/step - accuracy: 0.4373 - loss: 1.5039 - val_accuracy: 0.5223 - val_loss: 1.2917
Epoch 5/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 349s 862ms/step - accuracy: 0.4561 - loss: 1.4426 - val_accuracy: 0.5366 - val_loss: 1.2560
Epoch 6/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 347s 858ms/step - accuracy: 0.4718 - loss: 1.3958 - val_accuracy: 0.5471 - val_loss: 1.2254
Epoch 7/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 344s 852ms/step - accuracy: 0.4897 - loss: 1.3486 - val_accuracy: 0.5638 - val_loss: 1.2021
Epoch 8/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 359s 888ms/step - accuracy: 0.4998 -

# VGGFace - Ajuste fino

## Descongelar solo block5

In [13]:
vggface_finetune_model = load_model('/content/drive/MyDrive/FER-2013/modelos/vgg_feature_model.keras')

# En un modelo Sequential, el VGGFace está en la primera capa:
vggface_base = vggface_finetune_model.layers[0]

vggface_base.trainable = True

for layer in vggface_base.layers:
    if 'block5' not in layer.name:
        layer.trainable = False

## Compilación

In [14]:
vggface_finetune_model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Entrenamiento

In [15]:
history_vggface_finetune = vggface_finetune_model.fit(
    train_generator_vgg,      # RGB
    validation_data=val_generator_vgg,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

vggface_finetune_model.save('/content/drive/MyDrive/FER-2013/modelos/vggface_finetune_model.keras')

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 414s 963ms/step - accuracy: 0.6189 - loss: 1.0232 - val_accuracy: 0.6266 - val_loss: 1.0220
Epoch 2/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 348s 861ms/step - accuracy: 0.6257 - loss: 1.0062 - val_accuracy: 0.6259 - val_loss: 1.0224
Epoch 3/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 347s 860ms/step - accuracy: 0.6289 - loss: 0.9952 - val_accuracy: 0.6283 - val_loss: 1.0214
Epoch 4/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 346s 855ms/step - accuracy: 0.6279 - loss: 1.0063 - val_accuracy: 0.6311 - val_loss: 1.0220
Epoch 5/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 345s 853ms/step - accuracy: 0.6272 - loss: 0.9967 - val_accuracy: 0.6290 - val_loss: 1.0222
Epoch 6/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 345s 855ms/step - accuracy: 0.6167 - loss: 1.0222 - val_accuracy: 0.6301 - val_loss: 1.0224
Epoch 7/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 345s 855ms/step - accuracy: 0.6215 - loss: 1.0047 - val_accuracy: 0.6297 - val_loss: 1.0238
Epoch 8/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 347s 859ms/step - accuracy: 0.6277 -

# Evaluación usando sklearn.metrics (macro)

In [16]:
def evaluar_modelo(model, test_gen):
    y_true = test_gen.classes
    y_pred_probs = model.predict(test_gen)
    y_pred = np.argmax(y_pred_probs, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    return accuracy, precision, recall, f1

In [17]:
cnn_model = load_model('/content/drive/MyDrive/FER-2013/modelos/cnn_model.keras')
vgg_feature_model = load_model('/content/drive/MyDrive/FER-2013/modelos/vgg_feature_model.keras')
vggface_finetune_model = load_model('/content/drive/MyDrive/FER-2013/modelos/vggface_finetune_model.keras')

In [18]:
accuracy_cnn, precision_cnn, recall_cnn, f1_cnn = evaluar_modelo(cnn_model, test_generator)
accuracy_feat, precision_feat, recall_feat, f1_feat = evaluar_modelo(vgg_feature_model, test_generator_vgg)
accuracy_ft, precision_ft, recall_ft, f1_ft = evaluar_modelo(vggface_finetune_model, test_generator_vgg)

print(f"CNN: Accuracy={accuracy_cnn:.4f}, Precision={precision_cnn:.4f}, Recall={recall_cnn:.4f}, F1-score={f1_cnn:.4f}")
print(f"VGG Feature: Accuracy={accuracy_feat:.4f}, Precision={precision_feat:.4f}, Recall={recall_feat:.4f}, F1-score={f1_feat:.4f}")
print(f"VGG Fine-Tune: Accuracy={accuracy_ft:.4f}, Precision={precision_ft:.4f}, Recall={recall_ft:.4f}, F1-score={f1_ft:.4f}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 40s 351ms/step
113/113 ━━━━━━━━━━━━━━━━━━━━ 35s 299ms/step
CNN: Accuracy=0.5391, Precision=0.5204, Recall=0.4806, F1-score=0.4878
VGG Feature: Accuracy=0.6226, Precision=0.6256, Recall=0.5436, F1-score=0.5495
VGG Fine-Tune: Accuracy=0.6226, Precision=0.6208, Recall=0.5473, F1-score=0.5554
